In [1]:
import numpy as np
import os

# 导入matplotlib
import matplotlib.pyplot as plt

# 导入TF 2.0并设置GPU
import tensorflow as tf
# 导入Keras库
from tensorflow import keras
# 设置使用GPU显存的比例
# 对于RNN类型的模型，用GPU没有太大优势
from keras.backend.tensorflow_backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config)) 

# 导入TF相关模块
from tensorflow.keras import layers, losses, optimizers, Sequential
from tensorflow.keras.layers import Flatten, Dense, Embedding, SimpleRNN, LSTM, GRU

Using TensorFlow backend.


# 11. 循环神经网络
卷积神经网络利用数据的局部相关性和权值共享的思想大大减少了网络的参数量，非常适合于图片这种具有空间(Spatial)局部相关性的数据。

自然界的信号除了具有空间维度之外，还有一个时间(Temporal)维度，比如文本、语音信号、股市参数等。这类数据并不一定具有局部相关性，同时数据在时间维度上的长度也是可变的，卷积神经网络并不擅长处理此类数据。

本章介绍的循环神经网络可以较好地解决此类问题。

### 11.7 RNN短时记忆
循环神经网络除了训练困难，还有一个更严重的问题，那就是`短时记忆`(Short-term memory)。考虑一个长句子：

> 今天天气太美好了，尽管路上发生了一件不愉快的事情，…，我马上调整好状态，开开心心地准备迎接美好的一天。

根据我们的理解，之所以能够`开开心心地准备迎接美好的一天`，在于句子最开始处点名了`今天天气太美好了`。可见人类是能够很好地理解长句子的，但是，循环神经网络在处理较长的句子时，往往只能够理解有限长度内的信息，而对于位于较长范围类的有用信息往往不能够很好的利用起来。我们把这种现象叫做`短时记忆`。

针对这个问题，1997年，`Jürgen Schmidhuber`提出了`长短时记忆网络`(Long Short-Term Memory，简称`LSTM`)。LSTM相对于基础的RNN网络，记忆能力更强，更擅长处理较长的序列信号数据。LSTM广泛应用在序列预测、NLP等任务中，几乎取代了基础的RNN模型。


## 11.8 LSTM原理
基础的RNN网络结构如`图11.13`所示，上一个时间戳的状态向量$h_{t-1}$与当前时间戳的输入$x_t$经过线性变换后，通过激活函数`tanh`后得到新的状态向量$h_t$。相对于基础的RNN网络只有一个状态向量$h_t$ ，`LSTM`新增了一个状态向量$C_t$，同时引入了门控(Gate)机制，通过门控单元来控制信息的遗忘和刷新，如`图11.14`所示。

<img src="images/11_14.png" style="width:450px;"/>

在`LSTM`中，有两个状态向量$c$和$h$，其中$c$作为`LSTM`的内部状态向量，可以理解为`LSTM`的内存状态向量`Memory`，而$h$表示`LSTM`的输出向量。相对于基础的RNN来说，`LSTM`把内部`Memory`和输出分开为两个变量，同时利用三个门控：
+ 输入门(Input Gate)
+ 遗忘门(Forget Gate)
+ 输出门(Output Gate)

控制内部信息的流动。

门控机制可以理解为控制数据流通量的一种手段，类比于水阀门：当水阀门全部打开时，水流畅通无阻地通过；当水阀门全部关闭时，水流完全被隔断。在`LSTM`中，阀门开和程度利用门控值向量$g$表示，如`图11.15`所示。

<img src="images/11_15.png" style="width:250px;"/>

通过$\sigma{(g)}$激活函数将门控制压缩到$[0,1]$之间区间：
+ 当$\sigma{(g)} = 0$时，门控全部关闭，输出$o = 0$
+ 当$\sigma{(g)} = 1$时，门控全部打开，输出$o = x$

通过门控机制可以较好地控制数据的流量程度。

### 11.8.1 遗忘门
遗忘门作用于`LSTM`状态向量$c$上面，用于控制上一个时间戳的记忆$c_{t-1}$对当前时间戳的影响。遗忘门的控制变量$g_{f}$由
+ $g_f = \sigma(W_f[h_{t-1}, x_t] + b_f)$

产生，如`图11.16`所示。

<img src="images/11_16.png" style="width:350px;"/>

其中$W_f$和$b_f$为遗忘门的参数张量，可由反向传播算法自动优化，$\sigma$为激活函数，一般使用`Sigmoid`函数：
+ 当门控$g_{f} = 1$时，遗忘门全部打开，`LSTM`接受上一个状态$c_{t-1}$的所有信息
+ 当门控$g_{f} = 0$时，遗忘门关闭，`LSTM`直接忽略$c_{t-1}$，输出为0的向量

这也是遗忘门的名字由来。

经过遗忘门后，`LSTM`的状态向量变为$g_fc_{t−1}$。

### 11.8.2 输入门
输入门用于控制`LSTM`对输入的接收程度。首先通过对当前时间戳的输入$x_t$和上一个时间戳的输出$h_{t-1}$做非线性变换得到新的输入向量$\widetilde{c}_t$:
+ $\widetilde{c}_t = \tanh{(W_c[h_{t-1}, x_t] + b_c)}$

其中$W_c$和$b_c$为输入门的参数，需要通过反向传播算法自动优化，`tanh`为激活函数，用于将输入标准化到$[−1,1]$区间。$\widetilde{c}_t$并不会全部刷新进入`LSTM`的`Memory`，而是通过输入门控制接受输入的量。输入门的控制变量同样来自于输入$x_t$和输出$h_{t-1}$：
+ $g_i = \sigma(W_i[h_{t-1}, x_t] + b_i)$

其中$W_i$和$b_i$为输入门的参数，需要通过反向传播算法自动优化，$\sigma$为激活函数，一般使用`Sigmoid`函数。

输入门控制变量$g_i$决定了`LSTM`对当前时间戳的新输入$\widetilde{c}_t$的接受程度，如`图11.17`所示：

<img src="images/11_17.png" style="width:350px;"/>

+ 当$g_i = 0$时，`LSTM`不接受任何的新输入$\widetilde{c}_t$
+ 当$g_i = 1$时，`LSTM`全部接受新输入$\widetilde{c}_t$

经过输入门后，待写入`Memory`的向量为$g_i\widetilde{c}_t$。

### 11.8.3 刷新Memory
在遗忘门和输入门的控制下，`LSTM`有选择地读取了上一个时间戳的记忆$c_{t-1}$和当前时间戳的新输入$\widetilde{c}_t$，状态向量$c_t$的刷新方式为：
+ $c_t = g_i\widetilde{c}_t + g_fc_{t-1}$

得到的新状态向量$c_t$即为当前时间戳的状态向量，如`图11.17`所示。

### 11.8.4 输出门
`LSTM`的内部状态向量$c_{t}$并不会直接用于输出，这一点和基础的RNN不一样。基础的RNN网络的状态向量既用于记忆，又用于输出，所以基础的RNN可以理解为状态向量$c$和输出向量$h$是同一个对象。在`LSTM`内部，状态向量并不会全部输出，而是在输出门的作用下有选择地输出。输出门的门控变量$g_o$为：
+ $g_o = \sigma(W_O[h_{t-1},x_t]+b_o)$

其中$W_O$和$b_o$为输出门的参数，同样需要通过反向传播算法自动优化，$\sigma$为激活函数，一般使用`Sigmoid`函数：
+ 当输出门$g_o=0$时，输出关闭，`LSTM`的内部记忆完全被隔断，无法用作输出，此时输出为0的向量
+ 当输出门$g_o=1$时，输出完全打开，`LSTM`的状态向量$c_t$全部用于输出

`LSTM`的输出由：
+ $h_t = g_o\cdot\tanh(c_t)$

产生，即内存向量$c_t$经过`tanh`激活函数后与输入门作用，得到`LSTM`的输出。由于$g_o \in [0,1]，\tanh(c_t) \in [−1,1]$，因此`LSTM`的输出$t \in [−1,1]$。

<img src="images/11_18.png" style="width:350px;"/>

### 11.8.5 小结
`LSTM`虽然状态向量和门控数量较多，计算流程相对复杂。但是由于每个门控功能清晰明确，每个状态的作用也比较好理解。这里将典型的门控行为列举出来，并解释其代码的`LSTM`行为，如`表11.1`所示。

<img src="images/t_11_01.png" style="width:400px;"/>

## 11.9 LSTM层使用方法
`TensorFlow`有两种方式实现`LSTM`网络：
+ 使用`LSTMCell`手动完成时间戳上面的循环运算
+ 通过`LSTM`层方式一步完成前向运算

### 11.9.1 LSTMCell
`LSTMCell`的用法和`SimpleRNNCell`基本一致，区别在于`LSTM`的状态变量有两 个，即$[h_t,c_t]$，其中$h_t$为`cell`的输出，$c_t$为`cell`的更新后的状态。

首先新建一个状态向量长度$h=64$的`LSTMCell`，其中状态向量$c_t$和输出向量$h_t$的长度都为$h$：

In [2]:
x = tf.random.normal([2,80,100])
# 得到一个时间戳的输入
xt = x[:,0,:] 
# 创建LSTM Cell
cell = layers.LSTMCell(64) 
# 初始化状态和输出
state = [tf.zeros([2,64]),tf.zeros([2,64])]
# 前向计算
out, state = cell(xt, state) 
# 查看返回元素的 id
id(out),id(state[0]),id(state[1])

(140497035233424, 140497035233424, 140496971031136)

可以看到，返回的输出`out`和$c_t$的`id`是相同的，这与基础的`RNN`初衷一致，都是为了格式的统一。 通过在时间戳上展开循环运算，即可完成一次层的前向传播，写法与基础的`RNN`一样：

In [3]:
# 在序列长度维度上解开，循环送入LSTMCell单元
for xt in tf.unstack(x, axis=1):
    # 前向计算
    out, state = cell(xt, state)

输出可以仅使用最后一个时间戳上的输出，也可以聚合所有时间戳上的输出向量。

### 11.9.2 LSTM层

通过`layers.LSTM`层可以方便的一次完成整个序列的运算：

In [4]:
# 创建一层LSTM层，内存向量长度为64
layer = layers.LSTM(64)
# 序列通过LSTM层，默认返回最后一个时间戳的输出h
out = layer(x)
out.shape

TensorShape([2, 64])

经过`LSTM`层前向传播后，默认只会返回最后一个时间戳的输出，如果需要返回每个时间戳上面的输出，需要设置`return_sequences=True`标志：

In [5]:
# 创建LSTM层时，设置返回每个时间戳上的输出
layer = layers.LSTM(64, return_sequences=True)
# 前向计算，每个时间戳上的输出自动进行了concat，拼成一个张量
out = layer(x)
out.shape

TensorShape([2, 80, 64])

`out`的`shape`是$[2,80,64]$，包含了80个时间戳。

对于多层神经网络，可以通过`Sequential`容器包裹多层`LSTM`层，并设置所有非末层网络`return_sequences=True`，这是因为非末层的`LSTM`层需要上一层在所有时间戳的输出作为输入：

In [6]:
net = keras.Sequential([
    layers.LSTM(64, return_sequences=True),
    layers.LSTM(64)
])

out = net(x)
out.shape

TensorShape([2, 64])

## 11.10 GRU简介
`LSTM`具有更长的记忆能力，不容易出现梯度弥散现象，在大部分序列任务上面都取得了比基础的RNN模型更好的性能表现。但是`LSTM`结构相对较复杂，计算代价较高，模型参数量较大。因此，科学家们尝试简化`LSTM`内部的计算流程，特别是减少门控数量。研究发现，遗忘门是`LSTM`中最重要的门控，甚至发现只有遗忘门的简化版网络在多个基准数据集上面优于标准`LSTM`网络。

在众多的简化版`LSTM`中，`门控循环网络`(Gated Recurrent Unit，简称`GRU`)应用最广泛。`GRU`把内部状态向量和输出向量合并，统一为状态向量，门控数量也减少到2个：`复位门`(Reset Gate)和`更新门`(Update Gate)，如`图11.19`：

<img src="images/11_19.png" style="width:300px;"/>

### 11.10.1 复位门
复位门用于控制上一个时间戳的状态$h_{t-1}$进入`GRU`的量。门控向量$g_r$由当前时间戳输入$x_t$和上一时间戳状态$h_{t-1}$变换得到，关系如下：
+ $g_r = \sigma(W_r[h_{t-1},x_t]+b_r)$

其中$W_r$和$b_r$为复位门的参数，由反向传播算法自动优化，$\sigma$为激活函数，一般使用`Sigmoid`函数。门控向量$g_r$只控制状态$h_{t-1}$，而不会控制输入$x_t$：
+  $\tilde{h_t} = \tanh(W_h[g_rh_{t-1},x_t]+b_h)$

当$g_r = 0$时，新输入$\tilde{h_t}$全部来自于输入$x_t$，不接受$h_{t-1}$，此时相当于复位$h_{t-1}$。当$g_r = 1$时，$h_{t-1}$和输入$x_t$共同产生新输入$\tilde{h_t}$，如`图11.20`所示。

<img src="images/11_20.png" style="width:350px;"/>

### 11.10.2 更新门
更新门用控制上一时间戳状态$h_{t-1}$和新输入$\tilde{h_t}$对新状态向量$h_t$的影响程度。更新门控向量$g_z$由
+ $g_z = \sigma(W_z[h_{t-1}, x_t] + b_z)$

得到，其中$W_z$和$b_z$为更新门的参数，由反向传播算法自动优化，$\sigma$为激活函数，一般使用`Sigmoid`函数。$g_z$用与控制新输入$\tilde{h_t}$信号，$1-g_z$用于控制状态$h_{t-1}$信号：
+ $h_t = (1-g_z)h_{t-1}+g_z\tilde{h_t}$

可以看到，$\tilde{h_t}$和$h_{t-1}$对$h_t$的更新量处于相互竞争、此消彼长的状态：
+ 当更新门$g_z=0$时，$h_t$全部来自上一时间戳状态$h_{t-1}$
+ 当更新门$g_z = 1$时，$h_t$全部来自新输入$\tilde{h_t}$

<img src="images/11_21.png" style="width:350px;"/>

### 11.10.3 GRU使用方法
`TensorFlow`中，也有`Cell`方式和层方式实现`GRU`网络。`GRUCell`和`SimpleRNNCell`非常类似：

In [7]:
# 初始化状态向量，GRU只有一个
h = [tf.zeros([2,64])]
# 新建GRU Cell，向量长度为64
cell = layers.GRUCell(64)
# 在时间戳维度上解开，循环通过cell
for xt in tf.unstack(x, axis=1):
    out, h = cell(xt, h)
# 输出形状
out.shape

TensorShape([2, 64])

通过`layers.GRU`类可以方便创建一层`GRU`网络层，通过`Sequential`容器可以堆叠多层`GRU`层的网络：

In [8]:
net = keras.Sequential([
    layers.GRU(64, return_sequences=True),
    layers.GRU(64)
])
out = net(x)
out.shape

TensorShape([2, 64])

## 11.11 LSTM/GRU情感分类问题再战
首先准备数据：

In [9]:
# 批量大小
batchsz = 128 
# 词汇表大小N_vocab
total_words = 10000 
# 句子最大长度s，大于的句子部分将截断，小于的将填充
max_review_len = 80 
# 词向量特征长度f
embedding_len = 100 
# 加载IMDB数据集，此处的数据采用数字编码，一个数字代表一个单词
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=total_words)
print(x_train.shape, len(x_train[0]), y_train.shape)
print(x_test.shape, len(x_test[0]), y_test.shape)

(25000,) 218 (25000,)
(25000,) 68 (25000,)


In [10]:
# 截断和填充句子，使得等长，此处长句子保留句子后面的部分，短句子在前面填充
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_review_len)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_review_len)

# 构建数据集，打散，批量，并丢掉最后一个不够batchsz的batch
db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
db_train = db_train.shuffle(1000).batch(batchsz, drop_remainder=True)
db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
db_test = db_test.batch(batchsz, drop_remainder=True)
print('x_train shape:', x_train.shape, tf.reduce_max(y_train), tf.reduce_min(y_train))
print('x_test shape:', x_test.shape)

x_train shape: (25000, 80) tf.Tensor(1, shape=(), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
x_test shape: (25000, 80)


In [11]:
# 数字编码表
word_index = keras.datasets.imdb.get_word_index()
# for k,v in word_index.items():
#     print(k,v)
#%%
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

1646592/1641221 [==============================] - 2s 1us/step


### 11.11.1 LSTM模型
首先是`Cell`方式：

In [12]:
class MyRNN(keras.Model):
    # Cell方式构建多层网络
    def __init__(self, units):
        super(MyRNN, self).__init__()
        # [b, 64]，构建Cell初始化状态向量，重复使用
        self.state0 = [tf.zeros([batchsz, units]),tf.zeros([batchsz, units])]
        self.state1 = [tf.zeros([batchsz, units]),tf.zeros([batchsz, units])]
        # 词向量编码 [b, 80] => [b, 80, 100]
        self.embedding = layers.Embedding(total_words, embedding_len, input_length=max_review_len)
        # 构建2个Cell
        self.rnn_cell0 = layers.LSTMCell(units, dropout=0.5)
        self.rnn_cell1 = layers.LSTMCell(units, dropout=0.5)
        # 构建分类网络，用于将CELL的输出特征进行分类，2分类
        # [b, 80, 100] => [b, 64] => [b, 1]
        self.outlayer = Sequential([
        	layers.Dense(units),
        	layers.Dropout(rate=0.5),
        	layers.ReLU(),
        	layers.Dense(1)])

    def call(self, inputs, training=None):
        x = inputs # [b, 80]
        # embedding: [b, 80] => [b, 80, 100]
        x = self.embedding(x)
        # rnn cell compute,[b, 80, 100] => [b, 64]
        state0 = self.state0
        state1 = self.state1
        for word in tf.unstack(x, axis=1): # word: [b, 100] 
            out0, state0 = self.rnn_cell0(word, state0, training) 
            out1, state1 = self.rnn_cell1(out0, state1, training)
        # 末层最后一个输出作为分类网络的输入: [b, 64] => [b, 1]
        x = self.outlayer(out1,training)
        # p(y is pos|x)
        prob = tf.sigmoid(x)
        return prob

def main():
    units = 64 # RNN状态向量长度f
    epochs = 50 # 训练epochs

    model = MyRNN(units)
    # 装配
    # Keras在编译模型阶段，区分训练状态和非训练状态，二者的逻辑是不一样的。
    # 比如说，模型使用了Dropout，在训练时要随机失活部分神经元，而在正式运行，所有神经元都要保留的。
    # 如果是训练状态，则在编译参数里增加一个选项即可：
    # experimental_run_tf_function = False
    # 否则会报错：
    # _SymbolicException: Inputs to eager execution function cannot be Keras symbolic...
    model.compile(optimizer = optimizers.RMSprop(0.001),
                  loss = losses.BinaryCrossentropy(),
                  metrics=['accuracy'],
                  experimental_run_tf_function = False)
    # 训练和验证
    model.fit(db_train, epochs=epochs, validation_data=db_test)
    # 测试
    model.evaluate(db_test)

main()

Epoch 1/50
195/195 [==============================] - 25s 130ms/step - loss: 0.5117 - accuracy: 0.6468 - val_loss: 0.3664 - val_accuracy: 0.8392
Epoch 2/50
195/195 [==============================] - 12s 59ms/step - loss: 0.3534 - accuracy: 0.8451 - val_loss: 0.3550 - val_accuracy: 0.8431
Epoch 3/50
195/195 [==============================] - 11s 59ms/step - loss: 0.3061 - accuracy: 0.8719 - val_loss: 0.3577 - val_accuracy: 0.8432
Epoch 4/50
195/195 [==============================] - 11s 59ms/step - loss: 0.2845 - accuracy: 0.8883 - val_loss: 0.3602 - val_accuracy: 0.8457
Epoch 5/50
195/195 [==============================] - 12s 59ms/step - loss: 0.2637 - accuracy: 0.8960 - val_loss: 0.3711 - val_accuracy: 0.8445
Epoch 6/50
195/195 [==============================] - 12s 60ms/step - loss: 0.2491 - accuracy: 0.9040 - val_loss: 0.3631 - val_accuracy: 0.8454
Epoch 7/50
195/195 [==============================] - 12s 60ms/step - loss: 0.2384 - accuracy: 0.9069 - val_loss: 0.4336 - val_accuracy

Epoch 26/50
195/195 [==============================] - 11s 58ms/step - loss: 0.0662 - accuracy: 0.9753 - val_loss: 0.7841 - val_accuracy: 0.8077
Epoch 27/50
195/195 [==============================] - 11s 58ms/step - loss: 0.0595 - accuracy: 0.9776 - val_loss: 0.9759 - val_accuracy: 0.8137
Epoch 28/50
195/195 [==============================] - 11s 58ms/step - loss: 0.0548 - accuracy: 0.9803 - val_loss: 0.8539 - val_accuracy: 0.8188
Epoch 29/50
195/195 [==============================] - 11s 58ms/step - loss: 0.0510 - accuracy: 0.9827 - val_loss: 0.9302 - val_accuracy: 0.8159
Epoch 30/50
195/195 [==============================] - 11s 58ms/step - loss: 0.0481 - accuracy: 0.9828 - val_loss: 0.9425 - val_accuracy: 0.8127
Epoch 31/50
195/195 [==============================] - 11s 58ms/step - loss: 0.0418 - accuracy: 0.9847 - val_loss: 0.9962 - val_accuracy: 0.8147
Epoch 32/50
195/195 [==============================] - 11s 59ms/step - loss: 0.0432 - accuracy: 0.9845 - val_loss: 0.9764 - val_ac

195/195 [==============================] - 3s 16ms/step - loss: 1.4090 - accuracy: 0.8091


### 11.11.2 GRU模型
首先是`Cell`方式：

In [13]:
class MyRNN(keras.Model):
    # Cell方式构建多层网络
    def __init__(self, units):
        super(MyRNN, self).__init__()
        # [b, 64]，构建Cell初始化状态向量，重复使用
        self.state0 = [tf.zeros([batchsz, units])]
        self.state1 = [tf.zeros([batchsz, units])]
        # 词向量编码 [b, 80] => [b, 80, 100]
        self.embedding = layers.Embedding(total_words, embedding_len, input_length=max_review_len)
        # 构建2个Cell
        self.rnn_cell0 = layers.GRUCell(units, dropout=0.5)
        self.rnn_cell1 = layers.GRUCell(units, dropout=0.5)
        # 构建分类网络，用于将CELL的输出特征进行分类，2分类
        # [b, 80, 100] => [b, 64] => [b, 1]
        self.outlayer = Sequential([
        	layers.Dense(units),
        	layers.Dropout(rate=0.5),
        	layers.ReLU(),
        	layers.Dense(1)])

    def call(self, inputs, training=None):
        x = inputs # [b, 80]
        # embedding: [b, 80] => [b, 80, 100]
        x = self.embedding(x)
        # rnn cell compute,[b, 80, 100] => [b, 64]
        state0 = self.state0
        state1 = self.state1
        for word in tf.unstack(x, axis=1): # word: [b, 100] 
            out0, state0 = self.rnn_cell0(word, state0, training) 
            out1, state1 = self.rnn_cell1(out0, state1, training)
        # 末层最后一个输出作为分类网络的输入: [b, 64] => [b, 1]
        x = self.outlayer(out1, training)
        # p(y is pos|x)
        prob = tf.sigmoid(x)
        return prob

def main():
    units = 64 # RNN状态向量长度f
    epochs = 50 # 训练epochs
    model = MyRNN(units)
    # 装配
    model.compile(optimizer = optimizers.RMSprop(0.001),
                  loss = losses.BinaryCrossentropy(),
                  metrics=['accuracy'],
                  experimental_run_tf_function = False)
    # 训练和验证
    model.fit(db_train, epochs=epochs, validation_data=db_test)
    # 测试
    model.evaluate(db_test)

main()

Epoch 1/50
195/195 [==============================] - 32s 162ms/step - loss: 0.5494 - accuracy: 0.6126 - val_loss: 0.3693 - val_accuracy: 0.8348
Epoch 2/50
195/195 [==============================] - 13s 65ms/step - loss: 0.3665 - accuracy: 0.8360 - val_loss: 0.3447 - val_accuracy: 0.8476
Epoch 3/50
195/195 [==============================] - 13s 65ms/step - loss: 0.3231 - accuracy: 0.8659 - val_loss: 0.3516 - val_accuracy: 0.8457
Epoch 4/50
195/195 [==============================] - 13s 68ms/step - loss: 0.2901 - accuracy: 0.8823 - val_loss: 0.3668 - val_accuracy: 0.8445
Epoch 5/50
195/195 [==============================] - 13s 68ms/step - loss: 0.2689 - accuracy: 0.8938 - val_loss: 0.4414 - val_accuracy: 0.8336
Epoch 6/50
195/195 [==============================] - 13s 69ms/step - loss: 0.2498 - accuracy: 0.9001 - val_loss: 0.3700 - val_accuracy: 0.8445
Epoch 7/50
195/195 [==============================] - 14s 70ms/step - loss: 0.2335 - accuracy: 0.9105 - val_loss: 0.3859 - val_accuracy

Epoch 26/50
195/195 [==============================] - 13s 65ms/step - loss: 0.0668 - accuracy: 0.9772 - val_loss: 0.7020 - val_accuracy: 0.8205
Epoch 27/50
195/195 [==============================] - 13s 66ms/step - loss: 0.0611 - accuracy: 0.9786 - val_loss: 0.8931 - val_accuracy: 0.8158
Epoch 28/50
195/195 [==============================] - 13s 65ms/step - loss: 0.0579 - accuracy: 0.9801 - val_loss: 0.8520 - val_accuracy: 0.8145
Epoch 29/50
195/195 [==============================] - 13s 66ms/step - loss: 0.0506 - accuracy: 0.9824 - val_loss: 0.8845 - val_accuracy: 0.8175
Epoch 30/50
195/195 [==============================] - 13s 66ms/step - loss: 0.0435 - accuracy: 0.9854 - val_loss: 1.0585 - val_accuracy: 0.8168
Epoch 31/50
195/195 [==============================] - 13s 66ms/step - loss: 0.0458 - accuracy: 0.9847 - val_loss: 0.9253 - val_accuracy: 0.8159
Epoch 32/50
195/195 [==============================] - 13s 66ms/step - loss: 0.0424 - accuracy: 0.9864 - val_loss: 1.0935 - val_ac

195/195 [==============================] - 3s 17ms/step - loss: 1.4869 - accuracy: 0.8115


## 11.12 预训练的词向量
在情感分类任务时，`Embedding`层是从零开始训练的。实际上，对于文本处理任务来说，领域知识大部分是共享的，因此我们能够利用在其它任务上训练好的词向量。

我们以预训练的`GloVe`词向量为例，演示如何利用预训练的词向量模型提升任务性能。

In [14]:
embeddings_index = {}
GLOVE_DIR = r'/home/alex/datasets/glove'
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'),encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

len(embeddings_index), len(embeddings_index.keys()), len(word_index.keys())

(400000, 400000, 88588)

`GloVe.6B`版本共存储了40万个词汇的向量表。前面实战中我们只考虑最多1万个常见的词汇，我们根据词汇的数字编码表依次从GloVe模型中获取其词向量，并写入对应位置：

In [15]:
MAX_NUM_WORDS = total_words
# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, embedding_len))
applied_vec_count = 0
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    # print(word,embedding_vector)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        applied_vec_count += 1
print(applied_vec_count, embedding_matrix.shape)

9793 (10000, 100)


在获得了词汇表数据后，利用词汇表初始化`Embedding`层即可，并设置`Embedding`层不参与梯度优化：

In [18]:
class MyRNN(keras.Model):
    # Cell方式构建多层网络
    def __init__(self, units):
        super(MyRNN, self).__init__() 
        # 词向量编码 [b, 80] => [b, 80, 100]
        self.embedding = layers.Embedding(total_words, embedding_len,
                                          input_length=max_review_len,
                                          trainable=False)
        self.embedding.build(input_shape=(None,max_review_len))
        # self.embedding.set_weights([embedding_matrix])
        # 构建RNN
        self.rnn = keras.Sequential([
            layers.LSTM(units, dropout=0.5, return_sequences=True),
            layers.LSTM(units, dropout=0.5)
        ])
        # 构建分类网络，用于将CELL的输出特征进行分类，2分类
        # [b, 80, 100] => [b, 64] => [b, 1]
        self.outlayer = Sequential([
            layers.Dense(32),
            layers.Dropout(rate=0.5),
            layers.ReLU(),
            layers.Dense(1)])

    def call(self, inputs, training=None):
        x = inputs # [b, 80]
        # embedding: [b, 80] => [b, 80, 100]
        x = self.embedding(x)
        # rnn cell compute,[b, 80, 100] => [b, 64]
        x = self.rnn(x)
        # 末层最后一个输出作为分类网络的输入: [b, 64] => [b, 1]
        x = self.outlayer(x,training)
        # p(y is pos|x)
        prob = tf.sigmoid(x)
        return prob

def main():
    units = 512 # RNN状态向量长度f
    epochs = 50 # 训练epochs

    model = MyRNN(units)
    # 装配
    model.compile(optimizer = optimizers.Adam(0.001),
                  loss = losses.BinaryCrossentropy(),
                  metrics=['accuracy'],
                  experimental_run_tf_function = False)
    # 训练和验证
    model.fit(db_train, epochs=epochs, validation_data=db_test)
    # 测试
    model.evaluate(db_test)

main()

Epoch 1/50
195/195 [==============================] - 76s 388ms/step - loss: 0.6881 - accuracy: 0.5234 - val_loss: 0.6777 - val_accuracy: 0.5843
Epoch 2/50
195/195 [==============================] - 75s 385ms/step - loss: 0.6787 - accuracy: 0.5658 - val_loss: 0.6769 - val_accuracy: 0.5683
Epoch 3/50
195/195 [==============================] - 75s 387ms/step - loss: 0.6708 - accuracy: 0.5781 - val_loss: 0.6607 - val_accuracy: 0.5964
Epoch 4/50
195/195 [==============================] - 75s 386ms/step - loss: 0.6725 - accuracy: 0.5838 - val_loss: 0.6617 - val_accuracy: 0.5872
Epoch 5/50
195/195 [==============================] - 75s 387ms/step - loss: 0.6719 - accuracy: 0.5706 - val_loss: 0.6624 - val_accuracy: 0.5776
Epoch 6/50
195/195 [==============================] - 75s 387ms/step - loss: 0.6647 - accuracy: 0.5828 - val_loss: 0.6623 - val_accuracy: 0.5649
Epoch 7/50
195/195 [==============================] - 76s 388ms/step - loss: 0.6567 - accuracy: 0.5904 - val_loss: 0.6572 - val_ac

195/195 [==============================] - 74s 379ms/step - loss: 0.5418 - accuracy: 0.7051 - val_loss: 0.5246 - val_accuracy: 0.7248
Epoch 26/50
195/195 [==============================] - 74s 379ms/step - loss: 0.5421 - accuracy: 0.7104 - val_loss: 0.5510 - val_accuracy: 0.6737
Epoch 27/50
195/195 [==============================] - 74s 379ms/step - loss: 0.5344 - accuracy: 0.7116 - val_loss: 0.5356 - val_accuracy: 0.6960
Epoch 28/50
195/195 [==============================] - 74s 379ms/step - loss: 0.5307 - accuracy: 0.7176 - val_loss: 0.5226 - val_accuracy: 0.7271
Epoch 29/50
195/195 [==============================] - 74s 379ms/step - loss: 0.5280 - accuracy: 0.7229 - val_loss: 0.5649 - val_accuracy: 0.6606
Epoch 30/50
195/195 [==============================] - 74s 379ms/step - loss: 0.5230 - accuracy: 0.7195 - val_loss: 0.5308 - val_accuracy: 0.6963
Epoch 31/50
195/195 [==============================] - 74s 379ms/step - loss: 0.5210 - accuracy: 0.7221 - val_loss: 0.5151 - val_accurac

Epoch 50/50
195/195 [==============================] - 19s 99ms/step - loss: 0.4633 - accuracy: 0.7741


其它部分均保持一致。我们可以简单地比较通过预训练的 GloVe 模型初始化的 Embedding 层的训练结果和随机初始化的 Embedding 层的训练结果，在训练完 50 个 Epochs 后，预训 练模型的准确率达到了 84.7%，提升了约 2%。